In [8]:
import time
import os
import sys
import datetime
import pandas as pd
import pyarrow.parquet as pq
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import functions as F
import findspark
findspark.init()
findspark.find()

'C:\\Users\\re043430\\AppData\\Roaming\\Python\\Python39\\site-packages\\pyspark'

In [9]:
spark = SparkSession.builder \
    .master('local') \
    .appName('netFlixApp') \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [10]:
#Ler o Arquivo Parquet  .toDF('Title', 'Genre', 'GenreLabels', 'Premiere', 'Seasons', 'SeasonsParsed', 'EpisodesParsed', 'Length', 'MinLength', 'MaxLength', 'Status', 'Active', 'Table', 'Language', 'dt_inclusao')
df_NetFlix = spark.read.parquet('c:\\ortools\OriginaisNetflix.parquet') 

In [11]:
#Explorando o parquet importao para variável DF
df_NetFlix.show()

+--------------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------+---------+---------+----------+------+-----+--------+--------------------+
|               Title|               Genre|         GenreLabels| Premiere|             Seasons|SeasonsParsed|EpisodesParsed|     Length|MinLength|MaxLength|    Status|Active|Table|Language|         dt_inclusao|
+--------------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------+---------+---------+----------+------+-----+--------+--------------------+
|      House of Cards|     Political drama|     political,drama| 1-Feb-13|6 seasons, 73 epi...|            6|            73| 42–59 min.|       42|       59|     Ended|     0|Drama| English|2021-03-16T21:20:...|
|       Hemlock Grove|     Horror/thriller|     horror,thriller|19-Apr-13|3 seasons, 33 epi...|            3|            33| 45–58 min.|       45|       58|

In [12]:
#Convertendo os dados das Colunad 'Premiere' e 'dt_inclusão' de string para datetime
df_NetFlix1=df_NetFlix.withColumn('Premiere', F.from_unixtime(F.unix_timestamp('Premiere','d-MMM-yy')).cast('date')) \
    .withColumn('dt_inclusao', F.to_date(df_NetFlix.dt_inclusao).cast('date')) \

df_NetFlix1.show()

+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+-----------+---------+---------+----------+------+-----+--------+-----------+
|               Title|               Genre|         GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|     Length|MinLength|MaxLength|    Status|Active|Table|Language|dt_inclusao|
+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+-----------+---------+---------+----------+------+-----+--------+-----------+
|      House of Cards|     Political drama|     political,drama|2013-02-01|6 seasons, 73 epi...|            6|            73| 42–59 min.|       42|       59|     Ended|     0|Drama| English| 2021-03-16|
|       Hemlock Grove|     Horror/thriller|     horror,thriller|2013-04-19|3 seasons, 33 epi...|            3|            33| 45–58 min.|       45|       58|     Ended|     0|Drama| Englis

In [13]:
df_NetFlix2=df_NetFlix1.orderBy(df_NetFlix1.Active, df_NetFlix1.Genre)
df_NetFlix2.show(truncate=False)

+----------------------------------------------------+--------------------+--------------------+----------+----------------------+-------------+--------------+----------+---------+---------+----------+------+---------------+--------+-----------+
|Title                                               |Genre               |GenreLabels         |Premiere  |Seasons               |SeasonsParsed|EpisodesParsed|Length    |MinLength|MaxLength|Status    |Active|Table          |Language|dt_inclusao|
+----------------------------------------------------+--------------------+--------------------+----------+----------------------+-------------+--------------+----------+---------+---------+----------+------+---------------+--------+-----------+
|Blazing Transfer Students                           |Action comedy       |action,comedy       |2017-11-10|1 season, 8 episodes  |1            |8             |23–28 min.|23       |28       |Ended     |0     |ForeignLanguage|Japanese|2021-03-16 |
|The Day I Met E

In [14]:
df_NetFlix3=df_NetFlix2.dropDuplicates() \
    .withColumn('Seasons', F.regexp_replace('Seasons', 'TBA', 'a ser anunciado')) \
    .orderBy(F.desc("Active"),F.desc("Genre"))



df_NetFlix3.show()

+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+--------+--------+-----------+
|               Title|               Genre|         GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|   Table|Language|dt_inclusao|
+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+--------+--------+-----------+
|          True Tunes|chrildrens musica...|chrildrens,musica...|2019-07-12|     a ser anunciado|            0|             0|       TBA|        0|        0|Pending|     1|Children| English| 2021-03-16|
|Dance & Sing with...|chrildrens musica...|chrildrens,musica...|2018-05-18|1 season, 11 epis...|            1|            11|    2 min.|        2|        2|Pending|     1|Children| English| 20

In [15]:
df_NetFlix4=df_NetFlix3.withColumn('Data Alteração', F.current_timestamp().cast('date'))
df_NetFlix4.show()

+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+--------+--------+-----------+--------------+
|               Title|               Genre|         GenreLabels|  Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|   Table|Language|dt_inclusao|Data Alteração|
+--------------------+--------------------+--------------------+----------+--------------------+-------------+--------------+----------+---------+---------+-------+------+--------+--------+-----------+--------------+
|          True Tunes|chrildrens musica...|chrildrens,musica...|2019-07-12|     a ser anunciado|            0|             0|       TBA|        0|        0|Pending|     1|Children| English| 2021-03-16|    2022-11-27|
|Dance & Sing with...|chrildrens musica...|chrildrens,musica...|2018-05-18|1 season, 11 epis...|            1|            11|    2 m

In [16]:
df_NetFlix4.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: date (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: date (nullable = true)
 |-- Data Alteração: date (nullable = false)



In [17]:
df_NetFlix5=df_NetFlix4.withColumnRenamed('Title', 'Título') \
    .withColumnRenamed('Genre', 'Gênero') \
    .withColumnRenamed('GenreLabels', 'Rótulos de Gênero') \
    .withColumnRenamed('Premiere','Estreia') \
    .withColumnRenamed('Seasons','Temporada') \
    .withColumnRenamed('SeasonsParsed','Temporadas Anteriores') \
    .withColumnRenamed('EpisodesParsed','Episódios Anteriores') \
    .withColumnRenamed('Length','Duração') \
    .withColumnRenamed('MinLength','Menor Duração') \
    .withColumnRenamed('MaxLength','Maior Duração') \
    .withColumnRenamed('Status','Status') \
    .withColumnRenamed('Active','Ativa') \
    .withColumnRenamed('Table','Tabela') \
    .withColumnRenamed('Language','Idioma') \
    .withColumnRenamed('dt_inclusao','Data Inclusão') \
    .withColumnRenamed('datetime','Data e Hora')

df_NetFlix5.show(truncate=False)

+------------------------------------+------------------------+------------------------+----------+----------------------+---------------------+--------------------+----------+-------------+-------------+-------+-----+--------+-------+-------------+--------------+
|Título                              |Gênero                  |Rótulos de Gênero       |Estreia   |Temporada             |Temporadas Anteriores|Episódios Anteriores|Duração   |Menor Duração|Maior Duração|Status |Ativa|Tabela  |Idioma |Data Inclusão|Data Alteração|
+------------------------------------+------------------------+------------------------+----------+----------------------+---------------------+--------------------+----------+-------------+-------------+-------+-----+--------+-------+-------------+--------------+
|True Tunes                          |chrildrens musical short|chrildrens,musical,short|2019-07-12|a ser anunciado       |0                    |0                   |TBA       |0            |0            |P

In [ ]:
df_NetFlix5.write \
    .options('header', 'true') \
    ,options('sep', ';') \
    .csv('s3://repomaster')